In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [ ]:
#Loading dataset

In [ ]:
data=pd.read_csv("Pattuvam_data_till_Aug_2024_corrected.csv")
data

In [ ]:
data.dtypes

# Making Standardised Date Format

In [ ]:
from dateutil import parser
import pandas as pd

def standardize_date_format(date_str):
    try:
        # Use dateutil's parser to automatically recognize and convert various date formats
        date_obj = parser.parse(date_str)
        # Return the date in DD/MM/YYYY format
        return date_obj.strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        # Handle cases where date_str is not a recognizable date
        return 'Unknown'

# Assuming you have a DataFrame named data with a 'DATE' column
data['Standardized_Date'] = data['DATE'].apply(standardize_date_format)

print("DataFrame with standardized dates:")
data

# Making Standardised Time Format

In [ ]:
### For time standardisation

def standardize_time(time_str):
 
    time_str = time_str.replace(' ', ':')     #For Replacing the spaces with colons
    
    
        #For cases where time is not in HH:MM format
        
    if len(time_str) == 1:  # Single digit hour
        time_str = f'0{time_str}:00'
    elif len(time_str) == 2:  # Only hour 
        time_str = f'{time_str}:00'
    elif len(time_str) == 4:  # Format like HHMM
        time_str = f'{time_str[:2]}:{time_str[2:]}'
    elif len(time_str) == 5 and ':' not in time_str:  # Format like HHMM
        time_str = f'{time_str[:2]}:{time_str[2:]}'
    
    # Convert to datetime and format as HH:MM
    try:
        return pd.to_datetime(time_str, format='%H:%M', errors='coerce').strftime('%H:%M')
    except ValueError:
        return None  # Return None for unparseable formats

# Apply the function to the TIME column
data['STANDARDIZED_TIME'] = data['TIME'].apply(standardize_time)

In [ ]:
data.drop(index=range(2136, 2160), inplace=True) #These indexes shows 31/06/2024 data where June month contains only 30 days

In [ ]:
data['Standardized_Date'] = data['Standardized_Date'].str.replace('2922', '2022')
data['Standardized_Date'] = data['Standardized_Date'].str.replace('2032', '2022')
data['Standardized_Date'] = data['Standardized_Date'].str.replace('2044', '2022')

In [ ]:
data['Standardized_Date'] = pd.to_datetime(data['Standardized_Date'], format='DD/MM/YYYY',errors='coerce')

In [ ]:
data.to_csv('corrected_date_and_time.csv',index=False)

In [ ]:
#In this csv file some special characters are found in some of the column values and that cleaning is done manualy in the csv file. 
#So the after cleaning all the special characters the new csv file is All_Errors_Corrected.csv 

In [ ]:
# import numpy as np
# import pandas as pd
# import datetime
# import matplotlib.pyplot as plt
# import seaborn as sns
# import re
# from datetime import datetime

In [ ]:
data = pd.read_csv('All_Errors_Corrected.csv',parse_dates=['Standardized_Date'])
data.head()

# Date column cleaning

In [ ]:
# Correcting issue of 08-2022
data[data['Standardized_Date']=='2022-08-31']
# at index 18217 repetition started and ends at 18960
data.iloc[18216:18300]
data = data.drop(data.index[18217:18961])
data = data.reset_index(drop=True)
data.shape
# data[data['Standardized_Date']=='2022-08-31']
 
# # correcting the date where the two hours where misplaced
data[data['Standardized_Date']=='2022-08-20']
data.iloc[17927:17929]
# Replace the dates at indices 17927 and 17929 with the date from index 17926
data.loc[[17927, 17929], 'Standardized_Date'] = data.loc[17926,'Standardized_Date']
data.loc[[17928], 'STANDARDIZED_TIME'] = '1:00'
 
# # correcting date 2023-10-09 which was misplaced as 2023-10-10 
data[data['Standardized_Date']=='2023-10-11']
data.iloc[7440:7464]
# Replace the date in the column for the range of indices 7440 to 7464
data.loc[7440:7463, 'Standardized_Date'] = pd.Timestamp('2023-10-09')
 
# # 2023-11-09 is misplaced as 2023 -11-10 and 2023-11-10 is misplaced as 2023 -11-11
data[data['Standardized_Date']=='2023-11-10']
data.iloc[6720:6743]
data.loc[6720:6743, 'Standardized_Date'] = pd.Timestamp('2023-11-09')
data[data['Standardized_Date']=='2023-11-11']
data.iloc[6744:6767]
data.loc[6744:6767, 'Standardized_Date'] = pd.Timestamp('2023-11-10')
 
# # 2022-03-28 *48 since 2022 -05-28 is misplaced as 2022-03-2028
data[data['Standardized_Date']=='2022-03-28']
data.iloc[21790:21805]
# Replace the date in the column for the range of indices 20330 to 20354
data.loc[20329:20352, 'Standardized_Date'] = pd.Timestamp('2022-05-28')
 
# # 2022-05-16 (*25) since the 0:00 of  2022-05-15 was given this date,2023-05-14 the 0:00 of 2023-05-13 was given this date, the 0:00 of  2022-06-15 was given date 2022-06-16,same issue in 2023-01-07,same at 2022-07-16,same at 2022-12-07,same at 2022-04-16,same at 2022-12-01,same at 2022-03-16
# Loop through the DataFrame and check rows where time is '0:00'
 
for idx in data.index:
    if data.loc[idx, 'STANDARDIZED_TIME'] == '0:00':
        # Check the previous row's date and time
        if idx > 0:  # Ensure we are not checking the first row
            prev_date = data.loc[idx - 1, 'Standardized_Date']
            prev_time = data.loc[idx - 1, 'STANDARDIZED_TIME']
            # If the previous row's time is '23:00' and the dates are different
            if (prev_time == '23:00') and (prev_date != data.loc[idx, 'Standardized_Date']):
                # Reassign the current row's date to the previous date
                data.loc[idx, 'Standardized_Date'] = prev_date
# Dropping the error row with index 14544
data = data.drop(index=14544)
# Reset the index and drop the old index
data = data.reset_index(drop=True)
 
data[data['Standardized_Date']=='2022-08-28']
# Change the time at index 18126 to '7:00'
data.loc[18126, 'STANDARDIZED_TIME'] = '7:00'
 
#  2023-02-06 was also given to 3:00 of 2022-02-06
data[data['Standardized_Date']=='2022-02-06']
data.iloc[22005:22015]
data.loc[22009, 'Standardized_Date'] = pd.Timestamp('2022-02-06')
data.loc[22010, 'Standardized_Date'] = pd.Timestamp('2022-02-06')
 
# instead of 2023-08-10 the date 2023-10-08 is given at 20:00
data[data['Standardized_Date']=='2023-10-08']
data.loc[12619, 'Standardized_Date'] = pd.Timestamp('2023-08-10')
 
# 2022-06-04 was placed at 1:00 of 2022-07-04
data[data['Standardized_Date']=='2022-06-04']
data.loc[18288, 'Standardized_Date'] = pd.Timestamp('2022-07-04')
 
# 2023-03-20 at 2:00 was placed instead of 2023-03-21
data[data['Standardized_Date']=='2023-03-20']
data.loc[12121, 'Standardized_Date'] = pd.Timestamp('2023-03-21')
 
data[data['Standardized_Date']=='2023-12-03']
data.loc[11912, 'Standardized_Date'] = pd.Timestamp('2023-03-12')

In [ ]:
data.head()

# Remarks Cleaning

In [ ]:
data.REMARKS.unique()

In [ ]:
data['REMARKS'] = data['REMARKS'].str.lower()
Rem = data.REMARKS.value_counts().reset_index()
# Rem.index = Rem.index.lower
Rem

In [ ]:
data[data['REMARKS']=='maintenance work at']

In [ ]:
## REMARKS
#22697	2022-02-03	17:00	0.0	2.40	0.0	0.0	maintenance work at 
#22698	2022-02-03	18:00	0.0	2.09	0.0	0.0	intake

indices_maint = list(range(21952, 21954)) + \
          list(range(22187, 22189))
data.loc[indices_maint,'REMARKS'] = 'maintanance work at intake'
rows = data.loc[indices_maint]
rows

In [ ]:
list1 = ['1','3','pumping started','power failure at 7.28 to 7.36 pm','power failure at wtp 6.03 to6.40am']
for i in range(len(data)):
    if data.loc[i,'REMARKS'] in list1:
        data.loc[i,'REMARKS'] = np.nan

In [ ]:
data[(data['REMARKS']=='pumping stoped due to intake')|(data['REMARKS']=='pumping sopped due to inake')|(data['REMARKS']=='pumping stopped due to intake')]

In [ ]:
## REMARKS
# 19039	2022-07-04	7:00	0.0	2.94	0.00	0.0	pumping stoped due to intake
# 19040	2022-07-04	8:00	0.0	2.64	0.00	0.0	sump level low

indices_sump = list(range(18294, 18296)) + \
          list(range(18309, 18311)) + \
          list(range(18328, 18330)) + \
          list(range(18341, 18343)) + \
          list(range(18366, 18368))
data.loc[indices_sump,'REMARKS'] = 'intake sump level low'
rows = data.loc[indices_sump]
rows

In [ ]:
## REMARKS
# 17218	2022-09-20	10:00	0.0	3.38	0.0	0.0	pumping sopped due to inake
# 17219	2022-09-20	11:00	0.0	3.34	0.0	0.0	chamber cleaning

indices_chamber = list(range(17217, 17219))
data.loc[indices_chamber,'REMARKS'] = 'intake chamber cleaning'
rows = data.loc[indices_chamber]
rows

In [ ]:
data[data.REMARKS=='pumping stopped due to']

In [ ]:
## REMARKS
# 16257	2022-10-11	9:00	0.00	2.72	0.00	0.00	pumping stopped due to
# 16258	2022-10-11	10:00	0.00	2.66	0.00	0.00	intake chamber cleaning

indices_chamber = list(range(16256, 16258)) + \
          list(range(17264, 17266)) + \
          list(range(17913, 17915)) + \
          list(range(17937, 17939)) + \
          list(range(17959, 17961)) + \
          list(range(17986, 17988)) + \
          list(range(18033, 18035)) + \
          list(range(18057, 18059)) 
#           list(range(19289, 19291)) + \
#           list(range(21069, 21071)) + \
#           list(range(21103, 21105)) 
data.loc[indices_chamber,'REMARKS'] = 'intake chamber cleaning'
rows = data.loc[indices_chamber]
rows

In [ ]:
## REMARKS
# 20034	2022-06-14	18:00	0.0	3.12	0.0	0.0	pumping stopped due to
# 20035	2022-06-14	19:00	0.0	3.10	0.0	0.0	pipe line broken at taliparamba
# 21814	2022-04-27	22:00	NaN	NaN	NaN	0.0	pumping stopped due to
# 21815	2022-04-27	23:00	NaN	NaN	NaN	0.0	maintenance at thaliparamba

indices_thaliparambu =list(range(19289, 19291))
data.loc[indices_thaliparambu,'REMARKS'] = 'pipe line broken at Thaliparamba'

indices_thaliparambu =list(range(21069, 21071))
data.loc[indices_thaliparambu,'REMARKS'] = 'maintanance at Thaliparamba'


## REMARKS
# 21848	2022-04-29	8:00	0.0	2.0	0.0	0.0	pumping stopped due to
# 21849	2022-04-29	9:00	0.0	2.0	0.0	0.0	maintenance work at intake

indices_maintain =list(range(21103, 21105))
data.loc[indices_maintain,'REMARKS'] = 'maintanance work at intake'
rows = data.loc[indices_maintain]
rows

In [ ]:
data[data.REMARKS=='power failed @9.45 to 2.25']

In [ ]:
data.loc[13328,'REMARKS'] = np.nan
data.loc[13329,'REMARKS'] = 'power failure at intake and WTP'

In [ ]:
data[data.REMARKS=='permitted power']

In [ ]:
## REMARKS
# 23001	2022-02-16	9:00	3604.19	3.33	2781.45	3488.14	permitted power
# 23002	2022-02-16	10:00	0.00	0.00	0.00	0.00	failure at wtp

indices_wtp_power =list(range(22256, 22258)) 
data.loc[indices_wtp_power,'REMARKS'] = 'power failure at WTP'
data.loc[22256,'REMARKS'] = np.nan
rows = data.loc[indices_wtp_power]
rows

In [ ]:
data[data.REMARKS=='informed power failure at']

In [ ]:
## REMARKS
# 22497	2022-03-26	9:00	NaN	NaN	NaN	0.0	informed power failure at
# 22498	2022-03-26	10:00	NaN	NaN	NaN	0.0	intake

indices_intake_power = [21752,21753]
data.loc[indices_intake_power,'REMARKS'] = 'power failure at intake'
rows = data.loc[indices_intake_power]
rows

In [ ]:
data[data.REMARKS=='rw pumping stopped']

In [ ]:
data[5173:5175]

In [ ]:
data[data.REMARKS=='power failure']

In [ ]:
# 20938	2022-05-22	10:00	2475.82	3.92	2842.36	2396.10	power failure	
data.loc[20193,'REMARKS'] = np.nan

In [ ]:
data[5172:5195]

In [ ]:
# 5173	2024-01-06	14:00	1265.45	3.92	0.00	1224.70	rw pumping stopped
data.loc[5173,'REMARKS'] = np.nan
data.loc[5174,'REMARKS'] = 'intake pumping stopped'

In [ ]:
data['REMARKS'] = data['REMARKS'].str.lower()
Rem = data.REMARKS.value_counts().reset_index()
# Rem.index = Rem.index.lower
Rem

## Categorize REMARKS column

In [ ]:
df = data.copy()

In [ ]:
def categorize_remarks(data):
    """
    Categorizes the remarks column based on specific conditions:
    1. 'Power Failure at Intake' if:
       - Contains 'intake' (case-insensitive)
       - Contains 'power' (case-insensitive)
       - Does not contain 'wtp' (case-insensitive)
    
    2. 'Power Failure at WTP' if:
       - Does not contain 'intake'
       - Contains 'power' (case-insensitive)
       - Contains 'wtp' (case-insensitive)
    """
    
    def categorize(row):
        remarks = row['REMARKS']
        raw_water = row['RAW WATER FLOW IN m3/h']
        clear_water_pumping = row['CLEAR WATER PUMPING FLOW m3/h']
        if pd.notna(remarks):
            remarks_lower = remarks.lower()
            if ('intake' in remarks_lower and ('power' in remarks_lower or 'failure' in remarks_lower) and 'wtp' not in remarks_lower) or (('power' in remarks_lower or 'failure' in remarks_lower) and 'wtp' not in remarks_lower and (raw_water == 0.0 or pd.isna(raw_water)) and not pd.isna(clear_water_pumping) and clear_water_pumping != 0.0):
                return 'Power Failure at Intake'
            elif ('intake' not in remarks_lower and ('power' in remarks_lower or 'failure' in remarks_lower) and 'wtp' in remarks_lower) or (('power' in remarks_lower or 'failure' in remarks_lower) and 'intake' not in remarks_lower and not pd.isna(raw_water) and raw_water !=0.0 and (clear_water_pumping == 0.0 or pd.isna(clear_water_pumping))):
                return 'Power Failure at WTP'
            elif ('intake' in remarks_lower and ('power' in remarks_lower or 'failure' in remarks_lower) and 'wtp' in remarks_lower) or ('intake' not in remarks_lower and 'wtp' not in remarks_lower and ('power' in remarks_lower or 'failure' in remarks_lower) and (raw_water == 0.0 or pd.isna(raw_water)) and (clear_water_pumping == 0.0 or pd.isna(clear_water_pumping))):
                return 'Power Failure at intake and WTP'
            elif (('cleaning' in remarks_lower or 'wash' in remarks_lower) and 'intake' in remarks_lower):
                return 'intake cleaning'
            elif 'maint' in remarks_lower and 'intake' in remarks_lower:
                return 'maintanance at intake'
            elif 'maint' in remarks_lower and 'thaliparamba' in remarks_lower or 'taliparamba' in remarks_lower:
                return 'maintanance at Thaliparamba'
            elif 'maint' in remarks_lower and 'distribution' in remarks_lower:
                return 'distribution line maintanance'
            elif 'maint' in remarks_lower:
                return 'maintanance'
#             elif 'intake sump level low' in remarks_lower:
#                 return 'intake sump level low'
#             elif 'energy auditing' in remarks_lower:
#                 return 'energy auditing'
#             elif 'issue at thaliparambu' in remarks_lower:
#                 return 'issue at thaliparambu'
            elif 'cleaning' in remarks_lower and ('twr' in remarks_lower or 'wtp' in remarks_lower):
                return 'WTP cleaning'
#             elif 'intake pumping stopped' in remarks_lower:
#                 return 'intake pumping stopped'
            elif 'plc complaint' in remarks_lower:
                return 'complaint at WTP'
            else:
                return remarks_lower
            
#         return ''

    # Apply the categorization function to each row
    data['remarks category'] = data.apply(categorize, axis=1)
    
    return data


In [ ]:
df = categorize_remarks(df)
df

In [ ]:
df['remarks category'].value_counts()

In [ ]:
df[df['remarks category']=='maintanance']

In [ ]:
df[df['remarks category'].isna()].REMARKS.nunique()

In [ ]:
remain_list = list(df[df['remarks category'].isna()].REMARKS.unique())
remain_list

## Filling 'remarks category' column

In [ ]:
# Function to fill NaN values in 'remarks category' based on the conditions
def fill_remarks_category(df):
    previous_remark = None
#     remarks_flag=False
    for i in range(len(df)):
        current_remark = df.loc[i, 'remarks category']
        raw_water_flow = df.loc[i, 'RAW WATER FLOW IN m3/h']
        clear_water_pumping_flow = df.loc[i, 'CLEAR WATER PUMPING FLOW m3/h']
        # Check if 'remarks category' is NaN and if the previous row had a non-NaN remark
        if pd.isna(current_remark):
            # Check if raw water and treated water columns are 0 or NaN
            if ((raw_water_flow == 0 or pd.isna(raw_water_flow)) and (clear_water_pumping_flow == 0 or pd.isna(clear_water_pumping_flow))):# or (('intake' in previous_remark) and (raw_water_flow == 0) and (clear_water_pumping_flow != 0)) or (('WTP' in previous_remark) and (raw_water_flow != 0) and (clear_water_pumping_flow == 0)):
                # Fill the current NaN remark with the previous non-NaN remark
                df.loc[i, 'remarks category'] = previous_remark
            elif previous_remark is not None and (('intake' in previous_remark and 'WTP' not in previous_remark and (raw_water_flow == 0 or pd.isna(raw_water_flow)) and clear_water_pumping_flow != 0) or ('WTP' in previous_remark and 'intake' not in previous_remark and raw_water_flow != 0 and (clear_water_pumping_flow == 0 or pd.isna(clear_water_pumping_flow)))):
                df.loc[i, 'remarks category'] = previous_remark
            elif previous_remark is not None and 'complaint at WTP' in previous_remark and (raw_water_flow == 0 or pd.isna(raw_water_flow)):
                df.loc[i, 'remarks category'] = previous_remark
#             elif previous_remark is not None and 'WTP cleaning' in previous_remark and (raw_water_flow == 0 or pd.isna(raw_water_flow)) and (clear_water_flow == 0 or pd.isna(clear_water_flow)):
#                 df.loc[i, 'remarks category'] = previous_remark
            else:
                previous_remark = current_remark
        else:
            # Update the previous_remark if the current remark is not NaN
            previous_remark = current_remark
 
    return df
 


In [ ]:
# Apply the function to the DataFrame
df_filled = fill_remarks_category(df)

In [ ]:
df_filled[df_filled['remarks category']=='WTP cleaning']

In [ ]:
list1 = [22818,22819]
df_filled.loc[list1,'remarks category'] = None

In [ ]:
df_filled

In [ ]:
df_filled['remarks category'].value_counts()

In [ ]:
df_filled[df_filled['remarks category']=='maintanance'].REMARKS.unique()

In [ ]:
df_filled[df_filled.REMARKS=='stopped for maintanance']

In [ ]:
df_filled[12696:12716]

In [ ]:
df_filled[5150:5189]

In [ ]:
df_filled[df_filled['remarks category']=='complaint at WTP']

In [ ]:
df[6200:6234]

In [ ]:
df_filled[(df_filled['RAW WATER FLOW IN m3/h'].isna())&(df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna())&(df_filled['CLEAR WATER PUMPING FLOW m3/h'].isna())&(df_filled['TREATED WATER PRODUCTION IN m3/h']==0.0)&(df_filled['REMARKS'].isna())]

In [ ]:
df_filled[(df_filled['RAW WATER FLOW IN m3/h'].isna())&(df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna())&(df_filled['CLEAR WATER PUMPING FLOW m3/h'].isna())&(df_filled['TREATED WATER PRODUCTION IN m3/h']==0.0)&(df_filled['remarks category'].isna())]

In [ ]:
# df_filled.to_excel('df_filled.xlsx',index=False)

# Outlier Detection

In [ ]:
# Change the value at index 3355(13489.25)
df_filled.loc[3355, 'RAW WATER FLOW IN m3/h'] = 3489.25

# Change the value at index 6791(62411.58)
df_filled.loc[6791, 'RAW WATER FLOW IN m3/h'] = 2411.58

In [ ]:
def raw_water_outlier_ffill(data, column_name1,column_name2,column_name3, threshold=4000):
 
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name1] >= threshold:
            # Forward fill using the previous value
            data.loc[i, column_name1] = data.loc[i, column_name1] * 0.1
            
            data.loc[i,'TREATED WATER PRODUCTION IN m3/h'] = (data.loc[i, column_name1]*0.967).round(2)
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name2] > 5:
            # Forward fill using the previous value
            data.loc[i, column_name2] = data.loc[i-1, column_name2]
            
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name3] > threshold:
            
            # Forward fill using the previous value
            data.loc[i, column_name3] = data.loc[i-1, column_name3]
    return data

In [ ]:
df_filled = raw_water_outlier_ffill(df_filled, 'RAW WATER FLOW IN m3/h','CLEAR WATER SUMP LEVEL IN Meter','CLEAR WATER PUMPING FLOW m3/h')

# Null value cleaning

In [ ]:
df_filled.isna().sum()

In [ ]:
df_filled[(df_filled['RAW WATER FLOW IN m3/h'].isna()) & (df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna()) & (df_filled['CLEAR WATER PUMPING FLOW m3/h'].isna()) & (df_filled['TREATED WATER PRODUCTION IN m3/h']==0) & (df_filled['remarks category']).isna()]

In [ ]:
df_filled[df_filled['TREATED WATER PRODUCTION IN m3/h'].isna()]

In [ ]:
df_filled.loc[12619, 'TREATED WATER PRODUCTION IN m3/h'] = 3153.82

In [ ]:
df_filled[12618:12621]

In [ ]:
df_filled[df_filled['RAW WATER FLOW IN m3/h'].isna()]

In [ ]:
df_filled.loc[df_filled['remarks category'].notnull()] = df_filled.loc[df_filled['remarks category'].notnull()].fillna(0)

In [ ]:
df_filled[df_filled['RAW WATER FLOW IN m3/h'].isna()]

In [ ]:
df_filled.isna().sum()

In [ ]:
df_filled[df_filled['CLEAR WATER PUMPING FLOW m3/h'].isna()]

In [ ]:
indexes_to_fill = [2779, 8638, 8687, 23310]
df_filled.loc[indexes_to_fill, 'CLEAR WATER PUMPING FLOW m3/h'] = df_filled['CLEAR WATER PUMPING FLOW m3/h'].ffill()

In [ ]:
df_filled[df_filled['CLEAR WATER PUMPING FLOW m3/h'].isna()]

In [ ]:
df_filled[df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna()]

In [ ]:
indexes_to_fill = [321, 2604, 2605, 2606, 4911]
df_filled.loc[indexes_to_fill, 'CLEAR WATER SUMP LEVEL IN Meter'] = 0

In [ ]:
df_filled[df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna()]

In [ ]:
df_filled.isna().sum()

In [ ]:
df_filled.dtypes

In [ ]:
df_filled['STANDARDIZED_TIME'] = pd.to_datetime(df_filled['STANDARDIZED_TIME'],format='%H:%M').dt.time
df_filled=df_filled.sort_values(by=['Standardized_Date','STANDARDIZED_TIME'],ascending=True).reset_index(drop=True)

In [ ]:
df_filled.dtypes(#.dt.time attribute extracts time values as Python datetime.time objects, which are not one of the native Pandas types like
#datetime64 . Pandas treats datetime.time objects as object data type.)

In [ ]:
df_filled['REMARKS'].fillna('No Remarks', inplace=True)
df_filled['REMARKS'].replace(0,'No Remarks',inplace=True)
df_filled['remarks category'].fillna('No Remarks', inplace=True)

In [ ]:
df_filled[df_filled['CLEAR WATER SUMP LEVEL IN Meter'].isna()]

In [ ]:
df_filled[8735:8739]

In [ ]:
df_filled[15311:15314]

In [ ]:
indexes_to_fill = [8736,15312]
# # columns_to_fill = ['RAW WATER FLOW IN m3/h', 'CLEAR WATER PUMPING FLOW m3/h', 'CLEAR WATER SUMP LEVEL IN Meter']
# # df_filled.loc[indexes_to_fill, columns_to_fill] = df_filled[columns_to_fill].bfill()
df_filled.loc[indexes_to_fill] = df_filled.bfill()

In [ ]:
df_filled.isna().sum()

In [ ]:
flow_in_ML= ['RAW WATER FLOW IN m3/h','CLEAR WATER PUMPING FLOW m3/h',
       'TREATED WATER PRODUCTION IN m3/h']
df_filled[flow_in_ML]= df_filled[flow_in_ML]*0.001

In [ ]:
df_filled.rename(columns={'RAW WATER FLOW IN m3/h': 'RAW WATER FLOW IN ML','CLEAR WATER PUMPING FLOW m3/h':'CLEAR WATER PUMPING FLOW ML','TREATED WATER PRODUCTION IN m3/h':'TREATED WATER PRODUCTION IN ML'}, inplace=True)

In [ ]:
df_filled

In [ ]:
# df_filled.to_excel('final_pattuvam_cleaned_data.xlsx',index=False)

In [ ]:
df_filled.dtypes